In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from zipfile import ZipFile

path = r"/content/drive/MyDrive/Dataset/new_tomato_data.zip"

with ZipFile(path, 'r') as z:
  z.extractall()

In [3]:
import torch
import os
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import numpy as np

In [4]:
path = r'/content/new_tomato_data/train'
dataset = ImageFolder(path, transform = ToTensor())

In [5]:
img,label = dataset[1000]
print(img.dtype)
print(dataset.classes[label])
print(dataset.classes)

torch.float32
early_blight
['baterical_spot', 'early_blight', 'healthy', 'late_blight', 'leaf_mold', 'septoria_leaf_spot', 'spider_mites', 'target_spot', 'tomato_mosaic_virus', 'yellow_leaf_curl_virus']


In [6]:
train_ds, test_ds = random_split(dataset,[9500,500])
train_dl = DataLoader(train_ds, batch_size = 100)
test_dl = DataLoader(test_ds, 100)

In [7]:
import torch.nn as nn

In [8]:
class TomatoModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.network = nn.Sequential(
        nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),

        nn.MaxPool2d(2,2), #output 16 x 64 x 64
        
        nn.Conv2d(16,32,kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        
        nn.MaxPool2d(2,2), #output 32 x 32 x 32

        nn.Conv2d(32,64,kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        
        nn.MaxPool2d(2,2), #output 64 x 16 x 16

        nn.Conv2d(64,128,kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        
        nn.MaxPool2d(2,2), #output 128 x 8 x 8

        nn.Conv2d(128,256,kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        
        nn.MaxPool2d(2,2), #output 256 x 4 x 4

        nn.Flatten(),
        nn.Linear(256*4*4, 512),
        nn.ReLU(),
        
        nn.Linear(512, 64),
        nn.ReLU(),
        nn.Linear(64,10)
    )

  def forward(self,x):
    out = self.network(x)
    return torch.softmax(out, dim=-1)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
model = TomatoModel().to(device)
model.parameters()

<generator object Module.parameters at 0x7ff805ad29d0>

In [11]:
opt_func = torch.optim.SGD(model.parameters(), lr = 5e-1)
loss_func = nn.BCELoss()

In [15]:
num_epochs = 1

for epoch in range(num_epochs):
  for images, labels in train_dl:
    images = images.to(device)
    pred = model(images)
    result = []
    for label in labels:
      z = np.zeros(10)
      z[int(label)] = 1
      result.append(z.astype(np.float32))
    result = torch.tensor(result)
    result = result.to(device)
    loss = loss_func(pred, result)
    loss.backward()
    opt_func.step()
    opt_func.zero_grad()
  prediction_arr = torch.argmax(pred, 1)
  result_arr = torch.argmax(result, 1)
  accuracy = torch.sum(prediction_arr == result_arr).item()/len(prediction_arr)
  print(f'Accuracy: {accuracy*100} in the epoch: {epoch+1} and loss: {loss}')

Accuracy: 94.0 in the epoch: 1 and loss: 0.029911145567893982


In [17]:
torch.save(model.state_dict,'tomato.pth')